In [ ]:
!pip install pyvista

In [1]:
1-1

0

In [2]:
import numpy as np
import pyvista as pv

# NumPy array with shape (n_points, 3)
points = np.loadtxt('sample.xyz',skiprows=1)
point_cloud = pv.PolyData(points[:, :3])
mesh = point_cloud.reconstruct_surface()
mesh.save('mesh.stl')

In [1]:
import numpy as np
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
output_path="mesh"
dataname="sample.xyz"
point_cloud= np.loadtxt(dataname,skiprows=1)

In [3]:
point_cloud.shape

(1505010, 6)

In [3]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(point_cloud[:,:3])
pcd.colors = o3d.utility.Vector3dVector(point_cloud[:,3:6]/255)

In [4]:
pcd.estimate_normals(
    search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))

In [6]:
o3d.visualization.draw_geometries([pcd])

In [13]:
distances = pcd.compute_nearest_neighbor_distance()
avg_dist = np.mean(distances)
radius = 3 * avg_dist

In [ ]:
bpa_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(pcd,o3d.utility.DoubleVector([radius, radius * 2]))

In [ ]:
o3d.visualization.draw_geometries([bpa_mesh])

In [10]:
dec_mesh = bpa_mesh.simplify_quadric_decimation(100000)

In [11]:
dec_mesh.remove_degenerate_triangles()
dec_mesh.remove_duplicated_triangles()
dec_mesh.remove_duplicated_vertices()
dec_mesh.remove_non_manifold_edges()

TriangleMesh with 104561 points and 99986 triangles.

In [12]:
o3d.visualization.draw_geometries([dec_mesh])

In [ ]:
$Poisson reconstruction&

In [5]:
poisson_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=8, width=0, scale=1.1, linear_fit=False)[0]

In [6]:
bbox = pcd.get_axis_aligned_bounding_box()
p_mesh_crop = poisson_mesh.crop(bbox)

In [7]:
o3d.io.write_triangle_mesh(output_path+"bpa_mesh.ply", dec_mesh)
o3d.io.write_triangle_mesh(output_path+"p_mesh_c.ply", p_mesh_crop)

NameError: name 'dec_mesh' is not defined

In [8]:
o3d.visualization.draw_geometries([pcd])

In [13]:
poisson_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=10, width=0, scale=1.1, linear_fit=False)[0]

[WARNING] /root/Open3D/build/poisson/src/ext_poisson/PoissonRecon/Src/FEMTree.IsoSurface.specialized.inl (Line 1858)
          Extract
          bad average roots: 2


In [12]:
o3d.visualization.draw_geometries([poisson_mesh])

In [ ]:
bbox = pcd.get_axis_aligned_bounding_box()
p_mesh_crop = poisson_mesh.crop(bbox)

In [ ]:
o3d.io.write_triangle_mesh(output_path+"bpa_mesh.ply", dec_mesh)
o3d.io.write_triangle_mesh(output_path+"p_mesh_c.ply", p_mesh_crop)

In [ ]:
def lod_mesh_export(mesh, lods, extension, path):
    mesh_lods={}
    for i in lods:
        mesh_lod = mesh.simplify_quadric_decimation(i)
        o3d.io.write_triangle_mesh(path+"lod_"+str(i)+extension, mesh_lod)
        mesh_lods[i]=mesh_lod
    print("generation of "+str(i)+" LoD successful")
    return mesh_lods

In [ ]:
my_lods = lod_mesh_export(bpa_mesh, [100000,50000,10000,1000,100], ".ply", output_path)

In [ ]:
my_lods2 = lod_mesh_export(bpa_mesh, [8000,800,300], ".ply", output_path)

In [ ]:
o3d.visualization.draw_geometries([my_lods[10000]])